<a href="https://colab.research.google.com/github/ItaloNardy/NetworkAnalysis_U3/blob/main/ProjU3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyvis
!pip install wikipedia


from operator import itemgetter
from pyvis import network as net
import kagglehub
import pandas as pd
import networkx as nx
import wikipedia
import matplotlib.pyplot as plt
import seaborn as sns



nx.__version__

# Download latest version
path = kagglehub.dataset_download("tanishksharma9905/top-popular-anime")


print("Path to dataset files:", path)



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 54.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=e9409df82a05cab768269b8b380031682ac21337c338cc4cb0a7b99fc60fe61c
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia
Path to dataset files: /kaggle/input/top-popular-anime
